#setup

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
import spacy
import re
import math
import pickle
from itertools import product
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

from pathlib import Path
def yield_lines(filepath, gzipped=False, n_lines=None):
    filepath = Path(filepath)
    open_function = open
    if gzipped or filepath.name.endswith('.gz'):
        open_function = gzip.open
    with open_function(filepath, 'rt') as f:
        for i, l in enumerate(f):
            if n_lines is not None and i >= n_lines:
                break
            yield l.rstrip('\n')

def read_lines(filepath, gzipped=False):
    return list(yield_lines(filepath, gzipped=gzipped))

In [ ]:
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 50)

In [ ]:
cd '/content/drive/MyDrive/muss'

/content/drive/MyDrive/muss


In [ ]:
!pip install -e . &> /dev/null

# load wikilarge_detokenized

In [ ]:
# stop words
# https://gist.github.com/sebleier/554280
# nltk+SQL

stop_words_list = ["a","about","above","after","again","against","ain","all","am","an","and","any","are","aren","aren't","as","at","be","because","been","before","being","below","between","both","but","by","can","couldn","couldn't","d","did","didn","didn't","do","does","doesn","doesn't","doing","don","don't","down","during","each","few","for","from","further","had","hadn","hadn't","has","hasn","hasn't","have","haven","haven't","having","he","her","here","hers","herself","him","himself","his","how","i","if","in","into","is","isn","isn't","it","it's","its","itself","just","ll","m","ma","me","mightn","mightn't","more","most","mustn","mustn't","my","myself","needn","needn't","no","nor","not","now","o","of","off","on","once","only","or","other","our","ours","ourselves","out","over","own","re","s","same","shan","shan't","she","she's","should","should've","shouldn","shouldn't","so","some","such","t","than","that","that'll","the","their","theirs","them","themselves","then","there","these","they","this","those","through","to","too","under","until","up","ve","very","was","wasn","wasn't","we","were","weren","weren't","what","when","where","which","while","who","whom","why","will","with","won","won't","wouldn","wouldn't","y","you","you'd","you'll","you're","you've","your","yours","yourself","yourselves","could","he'd","he'll","he's","here's","how's","i'd","i'll","i'm","i've","let's","ought","she'd","she'll","that's","there's","they'd","they'll","they're","they've","we'd","we'll","we're","we've","what's","when's","where's","who's","why's","would","able","abst","accordance","according","accordingly","across","act","actually","added","adj","affected","affecting","affects","afterwards","ah","almost","alone","along","already","also","although","always","among","amongst","announce","another","anybody","anyhow","anymore","anyone","anything","anyway","anyways","anywhere","apparently","approximately","arent","arise","around","aside","ask","asking","auth","available","away","awfully","b","back","became","become","becomes","becoming","beforehand","begin","beginning","beginnings","begins","behind","believe","beside","besides","beyond","biol","brief","briefly","c","ca","came","cannot","can't","cause","causes","certain","certainly","co","com","come","comes","contain","containing","contains","couldnt","date","different","done","downwards","due","e","ed","edu","effect","eg","eight","eighty","either","else","elsewhere","end","ending","enough","especially","et","etc","even","ever","every","everybody","everyone","everything","everywhere","ex","except","f","far","ff","fifth","first","five","fix","followed","following","follows","former","formerly","forth","found","four","furthermore","g","gave","get","gets","getting","give","given","gives","giving","go","goes","gone","got","gotten","h","happens","hardly","hed","hence","hereafter","hereby","herein","heres","hereupon","hes","hi","hid","hither","home","howbeit","however","hundred","id","ie","im","immediate","immediately","importance","important","inc","indeed","index","information","instead","invention","inward","itd","it'll","j","k","keep","keeps","kept","kg","km","know","known","knows","l","largely","last","lately","later","latter","latterly","least","less","lest","let","lets","like","liked","likely","line","little","'ll","look","looking","looks","ltd","made","mainly","make","makes","many","may","maybe","mean","means","meantime","meanwhile","merely","mg","might","million","miss","ml","moreover","mostly","mr","mrs","much","mug","must","n","na","name","namely","nay","nd","near","nearly","necessarily","necessary","need","needs","neither","never","nevertheless","new","next","nine","ninety","nobody","non","none","nonetheless","noone","normally","nos","noted","nothing","nowhere","obtain","obtained","obviously","often","oh","ok","okay","old","omitted","one","ones","onto","ord","others","otherwise","outside","overall","owing","p","page","pages","part","particular","particularly","past","per","perhaps","placed","please","plus","poorly","possible","possibly","potentially","pp","predominantly","present","previously","primarily","probably","promptly","proud","provides","put","q","que","quickly","quite","qv","r","ran","rather","rd","readily","really","recent","recently","ref","refs","regarding","regardless","regards","related","relatively","research","respectively","resulted","resulting","results","right","run","said","saw","say","saying","says","sec","section","see","seeing","seem","seemed","seeming","seems","seen","self","selves","sent","seven","several","shall","shed","shes","show","showed","shown","showns","shows","significant","significantly","similar","similarly","since","six","slightly","somebody","somehow","someone","somethan","something","sometime","sometimes","somewhat","somewhere","soon","sorry","specifically","specified","specify","specifying","still","stop","strongly","sub","substantially","successfully","sufficiently","suggest","sup","sure","take","taken","taking","tell","tends","th","thank","thanks","thanx","thats","that've","thence","thereafter","thereby","thered","therefore","therein","there'll","thereof","therere","theres","thereto","thereupon","there've","theyd","theyre","think","thou","though","thoughh","thousand","throug","throughout","thru","thus","til","tip","together","took","toward","towards","tried","tries","truly","try","trying","ts","twice","two","u","un","unfortunately","unless","unlike","unlikely","unto","upon","ups","us","use","used","useful","usefully","usefulness","uses","using","usually","v","value","various","'ve","via","viz","vol","vols","vs","w","want","wants","wasnt","way","wed","welcome","went","werent","whatever","what'll","whats","whence","whenever","whereafter","whereas","whereby","wherein","wheres","whereupon","wherever","whether","whim","whither","whod","whoever","whole","who'll","whomever","whos","whose","widely","willing","wish","within","without","wont","words","world","wouldnt","www","x","yes","yet","youd","youre","z","zero","a's","ain't","allow","allows","apart","appear","appreciate","appropriate","associated","best","better","c'mon","c's","cant","changes","clearly","concerning","consequently","consider","considering","corresponding","course","currently","definitely","described","despite","entirely","exactly","example","going","greetings","hello","help","hopefully","ignored","inasmuch","indicate","indicated","indicates","inner","insofar","it'd","keep","keeps","novel","presumably","reasonably","second","secondly","sensible","serious","seriously","sure","t's","third","thorough","thoroughly","three","well","wonder"]

In [ ]:
# load dataset for preprocessing

PHASES = ['train','valid', 'test']
LANGUAGES = ['complex']
data_names = []

data_dir = '/content/drive/MyDrive/muss/resources/datasets/wikilarge_detokenized/'

for phase, language in product(PHASES, LANGUAGES):
  # print(phase, language)
  data_name = phase + '_' + language
  data_names.append(data_name)

  with open(data_dir+phase+'.'+language) as f:
        
    exec("%s = %s" % (data_name,f.read().splitlines()))

    f.close()

In [ ]:
# remove stop words
for index,file_name in enumerate(data_names):
    
    data = globals()[file_name]
    
    for count, sentence in enumerate(data):

        first_word = sentence.split(' ')[0]

        if first_word.lower() in stop_words_list:
            # delete it
            data[count] = sentence[len(first_word)+1:]

# operation preserving

## NER

In [ ]:
nlp = spacy.load("en_core_web_sm")

def substringSieve(NER_list,re_NER):
  string_list = []
  string_list.extend(NER_list)
  string_list.extend(re_NER)
  
  string_list.sort(key=lambda s: len(s), reverse=True)
  out = []
  for s in string_list:
      if not any([s in o for o in out]):
          out.append(s)
  
  return out

def find_all_NE(text):
  
  doc = nlp(text)
  NER_list = [t.text for t in doc.ents]
  re_NER = re.findall(r"[A-Z][A-z]*(?:(?=\s)\s[A-Z][A-z]*)*(?:(?!\s)\W[A-Z][A-z]*)*(?:(?:\sin(?=\s))*(?:\sof(?=\s))*(?:\sthe(?=\s))*(?:(?=\s[A-Z])\s[A-Z][A-z]*)*)*|(?:[0-9])*",text)
  re_NER = [x for x in re_NER if x != '']

  return substringSieve(NER_list,re_NER)

In [ ]:
data_names


['train_complex', 'valid_complex', 'test_complex']

In [ ]:
data_NER = []

for index,file_name in enumerate(data_names):
    
    data = globals()[file_name]
    
    found_NER_list = []

    for count, sentence in enumerate(data):
        
        found_NER_list += [find_all_NE(sentence)]

        if count%1000 == 0:
            print(file_name,count)
    
    data_NER += [found_NER_list]

# with open('/content/drive/MyDrive/muss/quantative/NER_list_wikilarge_0810_complex', 'wb') as fp:
#     pickle.dump(data_NER, fp)    

train_complex 0
train_complex 1000
train_complex 2000
train_complex 3000
train_complex 4000
train_complex 5000
train_complex 6000
train_complex 7000
train_complex 8000
train_complex 9000
train_complex 10000
train_complex 11000
train_complex 12000
train_complex 13000
train_complex 14000
train_complex 15000
train_complex 16000
train_complex 17000
train_complex 18000
train_complex 19000
train_complex 20000
train_complex 21000
train_complex 22000
train_complex 23000
train_complex 24000
train_complex 25000
train_complex 26000
train_complex 27000
train_complex 28000
train_complex 29000
train_complex 30000
train_complex 31000
train_complex 32000
train_complex 33000
train_complex 34000
train_complex 35000
train_complex 36000
train_complex 37000
train_complex 38000
train_complex 39000
train_complex 40000
train_complex 41000
train_complex 42000
train_complex 43000
train_complex 44000
train_complex 45000
train_complex 46000
train_complex 47000
train_complex 48000
train_complex 49000
train_complex

### delete stop words in found Named Entity

In [ ]:
# load NER found in train,valid,test set in Wikilarge

In [ ]:
with open ('/content/drive/MyDrive/muss/qualitative/NER_list_wikilarge_0803', 'rb') as fp:
    complex_data_NER = pickle.load(fp)

In [ ]:
with open ('/content/drive/MyDrive/muss/qualitative/NER_list_wikilarge_0807_simple', 'rb') as fp:
    simple_data_NER = pickle.load(fp)

In [ ]:
# define stop words list
cap_stop_words_list = [x.title() for x in stop_words_list]

In [ ]:
total_stop_words_list = []
total_stop_words_list.extend(cap_stop_words_list)
total_stop_words_list.extend(stop_words_list)
len(total_stop_words_list)

1568

In [ ]:
clean_complex_data_NER = []

for i in range(3):
  
  tmp_list = []
  for j in range(len(complex_data_NER[i])):

    tmp_list.append([x for x in complex_data_NER[i][j] if x not in cap_stop_words_list])
  
  clean_complex_data_NER += [tmp_list]

In [ ]:
clean_complex_data_NER[0][0:10]

[['James Edward Austen', 'the period 1809', 'Anna', '1814', '11'],
 ['Psalms of Thomas', 'Säve-Söderberg', "Mani's Psalms", 'Mandaean'],
 ['a period each year', 'six or three', 'Persephone', 'Hermes', 'Hades'],
 ['Cogeneration'],
 ['Switzerland', 'Romandie', 'Geneva', 'Zürich', 'French'],
 ['ten years later', 'Suzuka', 'Japan', 'F1'],
 ['its first month', 'August 1909'],
 ['the United Kingdom of Great Britain',
  'Hawaiian Independence Day',
  "The Kingdom of Hawai'i",
  'July Monarchy France',
  "Ka La Ku'oko '",
  'Ireland'],
 ['Oxford English Dictionary',
  'Oxford University Press',
  'the Grim Reaper',
  'mowing grass',
  'Scythe',
  '1933'],
 ['Toshihide Saito', 'Japanese']]

In [ ]:
clean_simple_data_NER = []

for i in range(3):

  tmp_list = []
  for j in range(len(simple_data_NER[i])):
    tmp_list.append([x for x in simple_data_NER[i][j] if x not in cap_stop_words_list])
  
  clean_simple_data_NER += [tmp_list]

In [ ]:
clean_simple_data_NER[0][0:10]

[['James Edward', 'Anna Austen', '1814'],
 ['Psalms of Thomas', 'Säve-Söderberg', "Mani's Psalms", 'Mandaean'],
 ['Underworld', 'Persephone', 'Demeter', 'Hades'],
 ['Cogeneration'],
 ['the Tribune de Genève',
  '1 February 1879',
  'James T. Bates',
  'about 187,000',
  'daily',
  'city'],
 ['ten years later', 'Suzuka', 'Japan', 'F1'],
 ['its first month', 'August 1909'],
 ['Hawaiian Independence Day',
  "The Kingdom of Hawai'i",
  'the United Kingdom',
  'November 28',
  'Ka La Ku',
  'France'],
 ['Oxford English Dictionary', 'Old English', '1933'],
 ['Toshihide Saito', 'April 1973', 'Japanese', '20']]

In [ ]:
with open('/content/drive/MyDrive/muss/qualitative/NER_clean_list_wikilarge_0810_complex', 'wb') as fp:
    pickle.dump(clean_complex_data_NER, fp)   

In [ ]:
with open('/content/drive/MyDrive/muss/qualitative/NER_clean_list_wikilarge_0810_simple', 'wb') as fp:
    pickle.dump(clean_simple_data_NER, fp)

## data cleansing

In [ ]:
with open ('/content/drive/MyDrive/muss/qualitative/NER_clean_list_wikilarge_0810_complex', 'rb') as fp:
    clean_complex_data_NER = pickle.load(fp)

In [ ]:
with open ('/content/drive/MyDrive/muss/qualitative/NER_clean_list_wikilarge_0810_simple', 'rb') as fp:
    clean_simple_data_NER = pickle.load(fp)

In [ ]:
with open ('/content/drive/MyDrive/muss/qualitative/NER_cooc_0823', 'rb') as fp:
    intersect_NE = pickle.load(fp)

In [ ]:
# load original sentence

data_names = []

PHASES = ['train', 'valid', 'test']
LANGUAGES = ['complex','simple']

data_dir = '/content/drive/MyDrive/muss/resources/datasets/wikilarge_detokenized/'

for phase, language in product(PHASES, LANGUAGES):
  # print(phase, language)
  data_name = phase + '_' + language + '_ori'
  data_names.append(data_name)
  with open(data_dir+phase+'.'+language) as f:
        
    exec("%s = %s" % (data_name,f.read().splitlines()))

    f.close()

In [ ]:
PHASES

['train', 'valid', 'test']

In [ ]:
for index,phase in enumerate(PHASES):
  data_tuples = list(zip(globals()[phase+'_complex_ori'],globals()[phase+'_simple_ori'],clean_complex_data_NER[index],clean_simple_data_NER[index],intersect_NE[index]))
  df_name = phase + '_df'
  print(df_name)
  globals()[df_name] = pd.DataFrame(data_tuples, columns=['complex','simple','complex_NER','simple_NER','co_occur_NER'])
  # file_name = data_name.replace('_','.')

train_df
valid_df
test_df


In [ ]:
train_df.drop(train_df[train_df.simple.map(lambda x: '.' not in x)].index, inplace=True)

In [ ]:
train_df.reset_index(inplace=True)

In [ ]:
c=0
for index, row in train_df.iterrows():
  if all([x.lower() not in row['complex'].lower() for x in row['simple'].split(' ')]):
    # print(row['complex'])
    # print(row['simple'])
    # print()
    train_df.drop(index, inplace=True)
    c+=1

In [ ]:
c

18182

In [ ]:
train_df.reset_index(inplace=True)

In [ ]:
train_df.to_pickle("/content/drive/MyDrive/muss/qualitative/final_wiki_train_df.pkl")

In [ ]:
valid_df.drop(valid_df[valid_df.simple.map(lambda x: '.' not in x)].index, inplace=True)

In [ ]:
valid_df.reset_index(inplace=True)

In [ ]:
c=0
for index, row in valid_df.iterrows():
  if all([x.lower() not in row['complex'].lower() for x in row['simple'].split(' ')]):
    valid_df.drop(index, inplace=True)
    c+=1

In [ ]:
c

60

In [ ]:
valid_df.reset_index(inplace=True)

In [ ]:
valid_df.to_pickle("/content/drive/MyDrive/muss/qualitative/final_wiki_valid_df.pkl")

In [ ]:
# dont need to clean test set as we dont use it in our work

In [ ]:
test_df.to_pickle("/content/drive/MyDrive/muss/qualitative/final_wiki_test_df.pkl")

## create training set (source sentence)

### NE are from complex sentence

In [ ]:
train_df = pd.read_pickle("/content/drive/MyDrive/muss/qualitative/final_wiki_train_df.pkl")
valid_df = pd.read_pickle("/content/drive/MyDrive/muss/qualitative/final_wiki_valid_df.pkl")
test_df = pd.read_pickle("/content/drive/MyDrive/muss/qualitative/final_wiki_test_df.pkl")

In [ ]:
target_dir = '/content/drive/MyDrive/muss/resources/datasets/wikilarge_final_complex_0821'
os.makedirs(target_dir)

In [ ]:
data_names = ['train_complex', 'valid_complex', 'test_complex']

In [ ]:
df_list = ['train_df','valid_df','test_df']

In [ ]:
for index,data_name in enumerate(data_names):

  chosen_df = globals()[df_list[index]]
  file_name = data_name.replace('_','.')

  with open(target_dir+'/'+file_name, 'w') as f:

    for index, row in chosen_df.iterrows():

        sentence = row.complex
        NER = row.complex_NER

        sentence_w_control_token = " <NEXT_NE> " +" <NEXT_NE> ".join(NER) + " <SENT_START> " + sentence

        if index%10000==0:
            print(index)
            print(sentence_w_control_token)

        f.write("{}\n".format(sentence_w_control_token))


  f.close()


0
 <NEXT_NE> James Edward Austen <NEXT_NE> the period 1809 <NEXT_NE> Anna <NEXT_NE> 1814 <NEXT_NE> 11 <SENT_START> There is manuscript evidence that Austen continued to work on these pieces as late as the period 1809 â "11, and that her niece and nephew, Anna and James Edward Austen, made further additions as late as 1814.
10000
 <NEXT_NE> the World Bank <NEXT_NE> Eurobonds <NEXT_NE> Lebanon <NEXT_NE> Syria <NEXT_NE> 1992 <SENT_START> In 1992, he became a prime minister, under Syria's watchful and military occupation of Lebanon, he put the country back on the financial map through the issuing of Eurobonds and won plaudits from the World Bank for his plan to borrow reconstruction money as the country's debt grew to become the largest per capita in the world.
20000
 <NEXT_NE> Kanpur Allahabad <NEXT_NE> GorakhpurAgra <NEXT_NE> Varanasi <NEXT_NE> Domestic <NEXT_NE> Lucknow <SENT_START> Domestic airports are Lucknow, GorakhpurAgra, Kanpur Allahabad and Varanasi.
30000
 <NEXT_NE> Major Gener

### NE are from simple sentence

In [ ]:
target_dir = '/content/drive/MyDrive/muss/resources/datasets/wikilarge_final_simple_0821'
os.makedirs(target_dir)

In [ ]:
data_names = ['train_complex', 'valid_complex', 'test_complex']

In [ ]:
df_list = ['train_df','valid_df','test_df']

In [ ]:
for index,data_name in enumerate(data_names):

  chosen_df = globals()[df_list[index]]
  file_name = data_name.replace('_','.')

  with open(target_dir+'/'+file_name, 'w') as f:

    for index, row in chosen_df.iterrows():

        sentence = row.complex
        # print(sentence)
        NER = row.simple_NER

        sentence_w_control_token = " <NEXT_NE> " +" <NEXT_NE> ".join(NER) + " <SENT_START> " + sentence

        if index%10000==0:
            print(index)
            print(sentence_w_control_token)

        f.write("{}\n".format(sentence_w_control_token))

  f.close()


0
 <NEXT_NE> James Edward <NEXT_NE> Anna Austen <NEXT_NE> 1814 <SENT_START> There is manuscript evidence that Austen continued to work on these pieces as late as the period 1809 â "11, and that her niece and nephew, Anna and James Edward Austen, made further additions as late as 1814.
10000
 <NEXT_NE> Prime Minister of Lebanon <NEXT_NE> 1992 <NEXT_NE> 1998 <NEXT_NE> 2000 <SENT_START> In 1992, he became a prime minister, under Syria's watchful and military occupation of Lebanon, he put the country back on the financial map through the issuing of Eurobonds and won plaudits from the World Bank for his plan to borrow reconstruction money as the country's debt grew to become the largest per capita in the world.
20000
 <NEXT_NE> Varanasi <NEXT_NE> Lucknow <NEXT_NE> Kanpur <NEXT_NE> Agra <SENT_START> Domestic airports are Lucknow, GorakhpurAgra, Kanpur Allahabad and Varanasi.
30000
 <NEXT_NE> Major General Chun Doo <NEXT_NE> December 1979 <NEXT_NE> Chun Doo-hwan <NEXT_NE> Choi <SENT_START> In

## dataset that contains NE appear in both sides 

### find co-occured NE

In [ ]:
 def substringSieve(NER_list,re_NER):
  string_list = []
  string_list.extend(NER_list)
  string_list.extend(re_NER)
  
  string_list.sort(key=lambda s: len(s), reverse=True)
  out = []
  for s in string_list:
      if not any([s in o for o in out]):
          out.append(s)



  return out

In [ ]:
def lcs(S,T):
    '''
    From https://www.bogotobogo.com/python/python_longest_common_substring_lcs_algorithm_generalized_suffix_tree.php
    '''
    m = len(S)
    n = len(T)
    counter = [[0]*(n+1) for x in range(m+1)]
    longest = 0
    lcs_set = set()
    for i in range(m):
        for j in range(n):
            if S[i] == T[j]:
                c = counter[i][j] + 1
                counter[i+1][j+1] = c
                if c > longest:
                    lcs_set = set()
                    longest = c
                    lcs_set.add(S[i-c+1:i+1])
                elif c == longest:
                    lcs_set.add(S[i-c+1:i+1])
    
    if len(lcs_set) == 1:# and ' ' not in list(lcs_set)[0]:
        return list(lcs_set)[0]
    else:
        return ''


In [ ]:
def get_NE_sent_intersect(a,simple_sent):
  # get the intersection of NE list and sentence
  output = []
  for index,item in enumerate(a):
    tmp_str = lcs(item,simple_sent)
    lower_tmp_str = lcs(item.lower(),simple_sent.lower())

    # print('| NE: item, tmp_str, lower_tmp_str\n')
    # print(item)
    # print(tmp_str)
    # print(lower_tmp_str)

    if tmp_str != '' and lower_tmp_str != '':

      if tmp_str[0] ==' ':
        tmp_str = tmp_str[1:]
        # print('modified tmp_str',tmp_str)

      if lower_tmp_str[0] ==' ':
        lower_tmp_str = lower_tmp_str[1:]
        # print('modified lower_tmp_str',lower_tmp_str)
    
      if tmp_str != '' and tmp_str[-1] ==' ':
        tmp_str = tmp_str[:-1]
        # print('modified tmp_str',tmp_str)
      if lower_tmp_str != '' and lower_tmp_str[-1] ==' ':
        lower_tmp_str = lower_tmp_str[:-1]
        # print('modified lower_tmp_str',lower_tmp_str)

      if len(tmp_str) != 0:
        if len(lower_tmp_str) == len(tmp_str)+1:
          # lower case NE exist in smiple sentence
          output.append(lower_tmp_str)
          # print('output',output)
        else:
          output.append(tmp_str)
          # print('output',output)

    # print('get_NE_sent_intersect output',output)
  
  return output


In [ ]:
def get_rid_of_subword(a,interset_list):
  # check if intersection is not a complete word in its original NE list
  output = []

  NE_lists = [x.split(' ') for x in a]
  list_lists = [item for sublist in NE_lists for item in sublist]
  
  # print('NE_lists',NE_lists)
  # print('list_lists',list_lists)

  for index,interset_item in enumerate(interset_list):
    # print('index,interset_item',index,interset_item)

    if any(x in interset_item for x in list_lists) or any(x.lower() in interset_item for x in list_lists):
      output.append(interset_item)
      # print('get_rid_of_subword output',output)

  return output


In [ ]:
stop_words_list = ["a","about","above","after","again","against","ain","all","am","an","and","any","are","aren","aren't","as","at","be","because","been","before","being","below","between","both","but","by","can","couldn","couldn't","d","did","didn","didn't","do","does","doesn","doesn't","doing","don","don't","down","during","each","few","for","from","further","had","hadn","hadn't","has","hasn","hasn't","have","haven","haven't","having","he","her","here","hers","herself","him","himself","his","how","i","if","in","into","is","isn","isn't","it","it's","its","itself","just","ll","m","ma","me","mightn","mightn't","more","most","mustn","mustn't","my","myself","needn","needn't","no","nor","not","now","o","of","off","on","once","only","or","other","our","ours","ourselves","out","over","own","re","s","same","shan","shan't","she","she's","should","should've","shouldn","shouldn't","so","some","such","t","than","that","that'll","the","their","theirs","them","themselves","then","there","these","they","this","those","through","to","too","under","until","up","ve","very","was","wasn","wasn't","we","were","weren","weren't","what","when","where","which","while","who","whom","why","will","with","won","won't","wouldn","wouldn't","y","you","you'd","you'll","you're","you've","your","yours","yourself","yourselves","could","he'd","he'll","he's","here's","how's","i'd","i'll","i'm","i've","let's","ought","she'd","she'll","that's","there's","they'd","they'll","they're","they've","we'd","we'll","we're","we've","what's","when's","where's","who's","why's","would","able","abst","accordance","according","accordingly","across","act","actually","added","adj","affected","affecting","affects","afterwards","ah","almost","alone","along","already","also","although","always","among","amongst","announce","another","anybody","anyhow","anymore","anyone","anything","anyway","anyways","anywhere","apparently","approximately","arent","arise","around","aside","ask","asking","auth","available","away","awfully","b","back","became","become","becomes","becoming","beforehand","begin","beginning","beginnings","begins","behind","believe","beside","besides","beyond","biol","brief","briefly","c","ca","came","cannot","can't","cause","causes","certain","certainly","co","com","come","comes","contain","containing","contains","couldnt","date","different","done","downwards","due","e","ed","edu","effect","eg","eight","eighty","either","else","elsewhere","end","ending","enough","especially","et","etc","even","ever","every","everybody","everyone","everything","everywhere","ex","except","f","far","ff","fifth","first","five","fix","followed","following","follows","former","formerly","forth","found","four","furthermore","g","gave","get","gets","getting","give","given","gives","giving","go","goes","gone","got","gotten","h","happens","hardly","hed","hence","hereafter","hereby","herein","heres","hereupon","hes","hi","hid","hither","home","howbeit","however","hundred","id","ie","im","immediate","immediately","importance","important","inc","indeed","index","information","instead","invention","inward","itd","it'll","j","k","keep","keeps","kept","kg","km","know","known","knows","l","largely","last","lately","later","latter","latterly","least","less","lest","let","lets","like","liked","likely","line","little","'ll","look","looking","looks","ltd","made","mainly","make","makes","many","may","maybe","mean","means","meantime","meanwhile","merely","mg","might","million","miss","ml","moreover","mostly","mr","mrs","much","mug","must","n","na","name","namely","nay","nd","near","nearly","necessarily","necessary","need","needs","neither","never","nevertheless","new","next","nine","ninety","nobody","non","none","nonetheless","noone","normally","nos","noted","nothing","nowhere","obtain","obtained","obviously","often","oh","ok","okay","old","omitted","one","ones","onto","ord","others","otherwise","outside","overall","owing","p","page","pages","part","particular","particularly","past","per","perhaps","placed","please","plus","poorly","possible","possibly","potentially","pp","predominantly","present","previously","primarily","probably","promptly","proud","provides","put","q","que","quickly","quite","qv","r","ran","rather","rd","readily","really","recent","recently","ref","refs","regarding","regardless","regards","related","relatively","research","respectively","resulted","resulting","results","right","run","said","saw","say","saying","says","sec","section","see","seeing","seem","seemed","seeming","seems","seen","self","selves","sent","seven","several","shall","shed","shes","show","showed","shown","showns","shows","significant","significantly","similar","similarly","since","six","slightly","somebody","somehow","someone","somethan","something","sometime","sometimes","somewhat","somewhere","soon","sorry","specifically","specified","specify","specifying","still","stop","strongly","sub","substantially","successfully","sufficiently","suggest","sup","sure","take","taken","taking","tell","tends","th","thank","thanks","thanx","thats","that've","thence","thereafter","thereby","thered","therefore","therein","there'll","thereof","therere","theres","thereto","thereupon","there've","theyd","theyre","think","thou","though","thoughh","thousand","throug","throughout","thru","thus","til","tip","together","took","toward","towards","tried","tries","truly","try","trying","ts","twice","two","u","un","unfortunately","unless","unlike","unlikely","unto","upon","ups","us","use","used","useful","usefully","usefulness","uses","using","usually","v","value","various","'ve","via","viz","vol","vols","vs","w","want","wants","wasnt","way","wed","welcome","went","werent","whatever","what'll","whats","whence","whenever","whereafter","whereas","whereby","wherein","wheres","whereupon","wherever","whether","whim","whither","whod","whoever","whole","who'll","whomever","whos","whose","widely","willing","wish","within","without","wont","words","world","wouldnt","www","x","yes","yet","youd","youre","z","zero","a's","ain't","allow","allows","apart","appear","appreciate","appropriate","associated","best","better","c'mon","c's","cant","changes","clearly","concerning","consequently","consider","considering","corresponding","course","currently","definitely","described","despite","entirely","exactly","example","going","greetings","hello","help","hopefully","ignored","inasmuch","indicate","indicated","indicates","inner","insofar","it'd","keep","keeps","novel","presumably","reasonably","second","secondly","sensible","serious","seriously","sure","t's","third","thorough","thoroughly","three","well","wonder"]
cap_stop_words_list = [x.title() for x in stop_words_list]
stop_words_all = []
stop_words_all.extend(stop_words_list)
stop_words_all.extend(cap_stop_words_list)
len(stop_words_all)

1568

In [ ]:
df_name = ['train_df','valid_df','test_df',]

In [ ]:
complex_ratio,simple_ratio,intersect_NE = [],[],[]

for i in range(3):
  tmp_complex_ratio = []
  tmp_simple_ratio = []
  tmp_NE = []

  for j in range(len(globals()[df_name[i]])):
    
    if j % 10000 == 0:
      print(i,j)

    a = globals()[df_name[i]].complex_NER[j].copy()
    b = globals()[df_name[i]].simple_NER[j].copy()

    complex_sent = globals()[df_name[i]].complex[j]
    simple_sent = globals()[df_name[i]].simple[j]
  
    # print(a,b)
    # print(complex_sent,'\n',simple_sent)

    # get the intersection of NE list and sentence
    complex_NE_intersect_simple = get_NE_sent_intersect(a,simple_sent)
    simple_NE_intersect_complex = get_NE_sent_intersect(b,complex_sent)

    # delete the intersection if is not a complete word in its original NE list
    # print('a,complex_NE_intersect_simple',a,complex_NE_intersect_simple)
    complex_NE_intersect_simple_final = get_rid_of_subword(a,complex_NE_intersect_simple)
    # print('b,simple_NE_intersect_complex',b,simple_NE_intersect_complex)
    simple_NE_intersect_complex_final = get_rid_of_subword(b,simple_NE_intersect_complex)

    # merge two lists into one and delete substring of any existing string
    output = substringSieve(complex_NE_intersect_simple_final,simple_NE_intersect_complex_final)
    # print('output',output)

    # delete stop words and capital stop words
    final = []
    for index,item in enumerate([x.split(' ') for x in output]):
      # if the item is 'WORD ' or ' WORD'

      # print('item,len(item)',item,len(item))
      if len(item) == 2 and '' in item:
        space_index = item.index('')
        str_index = 1 - space_index
        # dont append the word if it is a stop word
        if item[str_index] not in stop_words_all:
          final.append(item[str_index])
          # print('item[str_index]',item[str_index])

      # item is a phrase 
      else:
        clean_item = ' '.join(item)
        if clean_item[-1] == ' ':
          clean_item = clean_item[:-1]
        # print('clean_item',clean_item)
        final.append(clean_item)

    # print('final',final)
    
    # break down NE in complex_NE and simple_NE list to check if any NE is not included in the final output due to some special case such as
    '''
    complex sentence: In Holland they were called Stadspijpers, in Germany Stadtpfeifer and in Italy Pifferi.	
    simple sentence: They were called Stadtpfeifer in Germany and Pifferi in Italy.	
    NE in complex sentence: [Germany Stadtpfeifer, Italy Pifferi, Stadspijpers, Holland]
    NE in simple sentence: [Stadtpfeifer in Germany, Pifferi in Italy]
    
    the function lcs only look for the longest common substring, so when do lcs('Germany Stadtpfeifer',simple_sentence), the output is Stadtpfeifer.
    And when do lcs('Stadtpfeifer in Germany',complex_sentence), the output is also Stadtpfeifer.
    Hence 'Germany' is ignored in both cases.
    '''

    # split the item and flat the list of lists
    c = [item for sublist in [x.split(' ') for x in a] for item in sublist] # e.g.  ['Germany', 'Stadtpfeifer', 'Italy', 'Pifferi', 'Stadspijpers', 'Holland']
    d = [item for sublist in [x.split(' ') for x in b] for item in sublist] # e.g. ['Stadtpfeifer', 'in', 'Germany', 'Pifferi', 'in', 'Italy']
    
    intersection_1 = [x for ele in c for x in d if x in ele] # element in d that also is a (sub)string in c
    intersection_2 = [x for ele in d for x in c if x in ele]

    # do substringSieve twice
    final_output = substringSieve(substringSieve(intersection_1,intersection_2),final)
    # print('final_output',final_output)

    FINAL = [element for element in final_output if element not in stop_words_all]
    
    # print('FINAL',FINAL)
    ####################
    tmp_NE.append(FINAL)
    ####################

    # find the ratio of len(complex_NE_in_both)/len(complex) and the same for simple
    if len(FINAL) != 0:

      cc = list(set([element for element in c if element not in stop_words_all]))
      dd = list(set([element for element in d if element not in stop_words_all]))

      complex_in_inter_num = sum([x in FINAL for x in cc])
      # print('complex_in_inter_num',complex_in_inter_num)

      simple_in_inter_num = sum([x in FINAL for x in dd])
      # print('simple_in_inter_num',simple_in_inter_num)

      if len(a) != 0:
        complex_in_inter_ratio = complex_in_inter_num/len(a)
      else:
        complex_in_inter_ratio = np.nan

      if len(b) != 0:
        simple_in_inter_ratio = simple_in_inter_num/len(b)
      else:
        simple_in_inter_ratio = np.nan

      # print(complex_in_inter_ratio,simple_in_inter_ratio)
      ####################
      tmp_complex_ratio.append(complex_in_inter_ratio)
      tmp_simple_ratio.append(simple_in_inter_ratio)
      ####################

  complex_ratio += [tmp_complex_ratio]
  simple_ratio += [tmp_simple_ratio]
  intersect_NE += [tmp_NE]
    
  

0 0
0 10000
0 20000
0 30000
0 40000
0 50000
0 60000
0 70000
0 80000
0 90000
0 100000
0 110000
0 120000
0 130000
0 140000
0 150000
0 160000
0 170000
0 180000
0 190000
0 200000
0 210000
0 220000
0 230000
0 240000
1 0
2 0


In [ ]:
with open('/content/drive/MyDrive/muss/qualitative/NER_cooc_0823', 'wb') as fp:
    pickle.dump(intersect_NE, fp)

In [ ]:
with open('/content/drive/MyDrive/muss/qualitative/complex_ratio', 'wb') as fp:
    pickle.dump(complex_ratio, fp)

In [ ]:
with open('/content/drive/MyDrive/muss/qualitative/simple_ratio', 'wb') as fp:
    pickle.dump(simple_ratio, fp)

In [ ]:
with open ('/content/drive/MyDrive/muss/qualitative/NER_cooc_0823', 'rb') as fp:
    NER_cooc_0823 = pickle.load(fp)

In [ ]:
train_df['co_occur_NER'] = NER_cooc_0823[0]

In [ ]:
valid_df['co_occur_NER'] = NER_cooc_0823[1]

In [ ]:
test_df['co_occur_NER'] = NER_cooc_0823[2]

In [ ]:
with open('/content/drive/MyDrive/muss/qualitative/train_df_0911', 'wb') as fp:
    pickle.dump(train_df, fp)

In [ ]:
with open('/content/drive/MyDrive/muss/qualitative/valid_df_0911', 'wb') as fp:
    pickle.dump(valid_df, fp)

In [ ]:
with open('/content/drive/MyDrive/muss/qualitative/test_df_0911', 'wb') as fp:
    pickle.dump(test_df, fp)

### create dataset (source)

In [ ]:
target_dir = '/content/drive/MyDrive/muss/resources/datasets/wikilarge_cooc_0823'


In [ ]:
!mkdir $target_dir

In [ ]:
data_names = ['train_complex', 'valid_complex', 'test_complex']

In [ ]:
df_list = ['train_df','valid_df','test_df']

In [ ]:
for index,data_name in enumerate(data_names):

  chosen_df = globals()[df_list[index]]
  file_name = data_name.replace('_','.')

  with open(target_dir+'/'+file_name, 'w') as f:

    for index, row in chosen_df.iterrows():

        sentence = row.complex
        # print(sentence)
        NER = row.co_occur_NER

        sentence_w_control_token = " <NEXT_NE> " +" <NEXT_NE> ".join(NER) + " <SENT_START> " + sentence

        if index%10000==0:
            print(index)
            print(sentence_w_control_token)

        f.write("{}\n".format(sentence_w_control_token))


  f.close()


0
 <NEXT_NE> James Edward <NEXT_NE> Austen <NEXT_NE> Anna <NEXT_NE> 1814 <SENT_START> There is manuscript evidence that Austen continued to work on these pieces as late as the period 1809 â "11, and that her niece and nephew, Anna and James Edward Austen, made further additions as late as 1814.
10000
 <NEXT_NE> of Lebanon <NEXT_NE> 1992 <SENT_START> Red is also one of the subtractive primary colors of RYB color space but not CMYK color space.
20000
 <NEXT_NE> Varanasi <NEXT_NE> Lucknow <NEXT_NE> Kanpur <NEXT_NE> Agra <SENT_START> During ceremonies, the players covered their Reebok logos.
30000
 <NEXT_NE> Major General Chun Doo <NEXT_NE> December 1979 <NEXT_NE> Chun Doo-hwan <NEXT_NE> Choi <SENT_START> The first known rose garden was planted by empress JosÃ phine de Beauharnais in Malmaison.
40000
 <NEXT_NE> Braille <SENT_START> Most elements have more than one possible oxidation state.
50000
 <NEXT_NE>  <SENT_START> Billy Budd is an opera by Benjamin Britten, from a libretto by E. M. F

## create training set (target sentence)

In [ ]:
target_dir = '/content/drive/MyDrive/muss/resources/datasets/token_complex_wikilarge_0810'

In [ ]:
data_names = ['train_simple', 'valid_simple', 'test_simple']

In [ ]:
df_list = ['train_df','valid_df','test_df']

In [ ]:
for index,data_name in enumerate(data_names):

  chosen_df = globals()[df_list[index]]
  file_name = data_name.replace('_','.')

  with open(target_dir+'/'+file_name, 'w') as f:

    for index, row in chosen_df.iterrows():

        sentence = row.simple

        if index%10000==0:
            print(index)
            print(sentence)

        f.write("{}\n".format(sentence))

        # break

  f.close()


0
There is some proof that Austen continued to work on these pieces later in life. Her nephew and niece, James Edward and Anna Austen, may have made further additions to her work in around 1814.
10000
Red is the color of blood and tomatoes.
20000
1996 Atlanta Olympics; sprinter Linford Christie wore contact lenses embossed with the Puma AG logo at the press conference preceding the 100 metres final, despite Reebok being the official sponsor.
30000
The first known rose garden was planted by French empress JosÃ phine de Beauharnais in Malmaison.
40000
All elements have an oxidation state of 0.
50000
Billy Budd is an opera by Benjamin Britten. It was first performed at the Royal Opera House, Covent Garden, London on 1 December 1951.
60000
As such it is the subject of complex geopolitical issues, whereby providers must satisfy many legal restrictions in order to deliver service to a global market.
70000
Considered to be the turning point in the Ottoman Empires fortunes.
80000
Neigenfind, W

# operation lexical simplification

In [ ]:
train_df = pd.read_pickle("/content/drive/MyDrive/muss/qualitative/train_df_0911")
valid_df = pd.read_pickle("/content/drive/MyDrive/muss/qualitative/valid_df_0911")
test_df = pd.read_pickle("/content/drive/MyDrive/muss/qualitative/test_df_0911")

## training set

### obtain CERF level using Amplifi engine

In [ ]:
import requests
import json
import ast
from multiprocessing import Pool


In [ ]:
# API
url = #

headers = {
    'Content-Type': 'application/json',
    'x-api-key' : #
}

In [ ]:
# request word level difficulty 
def get_data(text):
    r1 = requests.request("POST", url, headers=headers, data=json.dumps([{'id':'test','text':text}]))
    a = r1.json()
    return a

In [ ]:
# get results for trainning set

In [ ]:
with Pool(5) as p:
    result = p.map(get_data, train_df['complex'])

with open('/content/drive/MyDrive/muss/qualitative/train_complex_ABCD', 'wb') as fp:
    pickle.dump(result, fp)

In [ ]:
with Pool(5) as p:
    result = p.map(get_data, train_df['simple'])

with open('/content/drive/MyDrive/muss/qualitative/train_simple_ABCD', 'wb') as fp:
    pickle.dump(result, fp)

In [ ]:
with open('/content/drive/MyDrive/muss/qualitative/train_complex_ABCD', 'rb') as f:
    train_complex_result = pickle.load(f)
with open('/content/drive/MyDrive/muss/qualitative/train_simple_ABCD', 'rb') as f:
    train_simple_result = pickle.load(f)

In [ ]:
# combine results for complex and simple sentences
from pandas import json_normalize

def process_file(f):
  jsonObj = json.loads(f)
  return json_normalize(jsonObj)

In [ ]:
frames = []

for i in range(len(train_complex_result)):

  if 'body' not in train_complex_result[i].keys():
    print('error',i)
    tmp = process_file(train_complex_result[i-1]['body'])
    tmp.loc[:] = np.nan
    frames.append(tmp)
  else:
    frames.append(process_file(train_complex_result[i]['body']))

train_complex_df = pd.concat(frames)

with open('/content/drive/MyDrive/muss/qualitative/ABCD_train_complex', 'wb') as fp:
    pickle.dump(train_complex_df, fp)    

In [ ]:
frames = []

for i in range(len(train_simple_result)):

  if 'body' not in train_simple_result[i].keys():
    print('error',i)
    tmp = process_file(train_simple_result[i-1]['body'])
    tmp.loc[:] = np.nan
    frames.append(tmp)
  else:
    frames.append(process_file(train_simple_result[i]['body']))

train_simple_df = pd.concat(frames)

with open('/content/drive/MyDrive/muss/qualitative/ABCD_train_simple', 'wb') as fp:
    pickle.dump(train_simple_df, fp)    

In [ ]:
with open('/content/drive/MyDrive/muss/qualitative/ABCD_train_complex', 'rb') as f:
    train_complex_df = pickle.load(f)
with open('/content/drive/MyDrive/muss/qualitative/ABCD_train_simple', 'rb') as f:
    train_simple_df = pickle.load(f)
    

In [ ]:
new_train_complex_df = train_complex_df.drop(columns=['id','read.A1','read.B1','read.B2','read.C1','read.overall']).add_prefix('complex_')
new_train_simple_df = train_simple_df.drop(columns=['id','read.A1','read.B1','read.B2','read.C1','read.overall']).add_prefix('simple_')
combined_df = pd.concat([new_train_complex_df, new_train_simple_df], axis=1).reset_index().drop(columns='index')
combined_df

,complex_read_level,complex_words,simple_read_level,simple_words
0,C1,"[{'id': 1, 'text': 'There', 'start_idx': 0, 'e...",B2,"[{'id': 1, 'text': 'There', 'start_idx': 0, 'e..."
1,B2,"[{'id': 1, 'text': 'In', 'start_idx': 0, 'end_...",B2,"[{'id': 1, 'text': 'Mandaean', 'start_idx': 0,..."
2,B2,"[{'id': 1, 'text': 'Before', 'start_idx': 0, '...",C1,"[{'id': 1, 'text': 'When', 'start_idx': 0, 'en..."
3,B2,"[{'id': 1, 'text': 'Cogeneration', 'start_idx'...",B2,"[{'id': 1, 'text': 'Cogeneration', 'start_idx'..."
4,B1,"[{'id': 1, 'text': 'Geneva', 'start_idx': 0, '...",C1,"[{'id': 1, 'text': 'The', 'start_idx': 0, 'end..."
...,...,...,...,...
249161,C2,"[{'id': 1, 'text': 'These', 'start_idx': 0, 'e...",C2,"[{'id': 1, 'text': 'These', 'start_idx': 0, 'e..."
249162,C2,"[{'id': 1, 'text': 'Saint', 'start_idx': 0, 'e...",C2,"[{'id': 1, 'text': 'Saint', 'start_idx': 0, 'e..."
249163,B2,"[{'id': 1, 'text': 'The', 'start_idx': 0, 'end...",A1,"[{'id': 1, 'text': 'There', 'start_idx': 0, 'e..."
249164,C1,"[{'id': 1, 'text': 'Boulogne', 'start_idx': 0,...",C2,"[{'id': 1, 'text': 'Consequently', 'start_idx'..."


### find C1+C2+B2 words


In [ ]:
sentence_df = train_df.drop(columns=['level_0','index','complex_NER','simple_NER','co_occur_NER'])
sentence_df

,complex,simple
0,There is manuscript evidence that Austen conti...,There is some proof that Austen continued to w...
1,"In a remarkable comparative analysis, Mandaean...",Mandaean scholar Säve-Söderberg showed that Ma...
2,"Before Persephone was released to Hermes, who ...",When Demeter went to the Underworld to rescue ...
3,Cogeneration plants are commonly found in dist...,Cogeneration plants are commonly found in dist...
4,"Geneva (,;,;,;;) is the second-most-populous c...",The city's main newspaper is the Tribune de Ge...
...,...,...
249161,These structures form at the high-pressure dep...,These structures form at the high-pressure dep...
249162,Saint-Maixant is a commune in the Gironde depa...,"Saint-Maixant, Gironde is a commune. It is fou..."
249163,The Hollywood Walk of Fame is a series of side...,There are many stars on the sidewalk.
249164,Boulogne-Billancourt is the most populous subu...,"Consequently, Boulogne-Billancourt is one of t..."


In [ ]:
FINAL_df = pd.concat([sentence_df, combined_df], axis=1)
FINAL_df = FINAL_df[FINAL_df['complex_words'].notna()][FINAL_df['simple_words'].notna()]
FINAL_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,complex,simple,complex_read_level,complex_words,simple_read_level,simple_words
0,There is manuscript evidence that Austen conti...,There is some proof that Austen continued to w...,C1,"[{'id': 1, 'text': 'There', 'start_idx': 0, 'e...",B2,"[{'id': 1, 'text': 'There', 'start_idx': 0, 'e..."
1,"In a remarkable comparative analysis, Mandaean...",Mandaean scholar Säve-Söderberg showed that Ma...,B2,"[{'id': 1, 'text': 'In', 'start_idx': 0, 'end_...",B2,"[{'id': 1, 'text': 'Mandaean', 'start_idx': 0,..."
2,"Before Persephone was released to Hermes, who ...",When Demeter went to the Underworld to rescue ...,B2,"[{'id': 1, 'text': 'Before', 'start_idx': 0, '...",C1,"[{'id': 1, 'text': 'When', 'start_idx': 0, 'en..."
3,Cogeneration plants are commonly found in dist...,Cogeneration plants are commonly found in dist...,B2,"[{'id': 1, 'text': 'Cogeneration', 'start_idx'...",B2,"[{'id': 1, 'text': 'Cogeneration', 'start_idx'..."
4,"Geneva (,;,;,;;) is the second-most-populous c...",The city's main newspaper is the Tribune de Ge...,B1,"[{'id': 1, 'text': 'Geneva', 'start_idx': 0, '...",C1,"[{'id': 1, 'text': 'The', 'start_idx': 0, 'end..."
...,...,...,...,...,...,...
249161,These structures form at the high-pressure dep...,These structures form at the high-pressure dep...,C2,"[{'id': 1, 'text': 'These', 'start_idx': 0, 'e...",C2,"[{'id': 1, 'text': 'These', 'start_idx': 0, 'e..."
249162,Saint-Maixant is a commune in the Gironde depa...,"Saint-Maixant, Gironde is a commune. It is fou...",C2,"[{'id': 1, 'text': 'Saint', 'start_idx': 0, 'e...",C2,"[{'id': 1, 'text': 'Saint', 'start_idx': 0, 'e..."
249163,The Hollywood Walk of Fame is a series of side...,There are many stars on the sidewalk.,B2,"[{'id': 1, 'text': 'The', 'start_idx': 0, 'end...",A1,"[{'id': 1, 'text': 'There', 'start_idx': 0, 'e..."
249164,Boulogne-Billancourt is the most populous subu...,"Consequently, Boulogne-Billancourt is one of t...",C1,"[{'id': 1, 'text': 'Boulogne', 'start_idx': 0,...",C2,"[{'id': 1, 'text': 'Consequently', 'start_idx'..."


In [ ]:
with open('/content/drive/MyDrive/muss/qualitative/ABCD_train_FINAL', 'wb') as fp:
    pickle.dump(FINAL_df, fp)

In [ ]:
FINAL_df = pd.concat([train_df.drop(columns=['level_0','index']), FINAL_df.drop(columns=['complex','simple'])],axis=1)
FINAL_df

,complex,simple,complex_NER,simple_NER,co_occur_NER,complex_read_level,complex_words,simple_read_level,simple_words
0,There is manuscript evidence that Austen conti...,There is some proof that Austen continued to w...,"[James Edward Austen, the period 1809, Anna, 1...","[James Edward, Anna Austen, 1814]","[James Edward, Austen, Anna, 1814]",C1,"[{'id': 1, 'text': 'There', 'start_idx': 0, 'e...",B2,"[{'id': 1, 'text': 'There', 'start_idx': 0, 'e..."
1,"In a remarkable comparative analysis, Mandaean...",Mandaean scholar Säve-Söderberg showed that Ma...,"[Psalms of Thomas, Säve-Söderberg, Mani's Psal...","[Psalms of Thomas, Säve-Söderberg, Mani's Psal...","[Psalms of Thomas, Säve-Söderberg, Mani's Psal...",B2,"[{'id': 1, 'text': 'In', 'start_idx': 0, 'end_...",B2,"[{'id': 1, 'text': 'Mandaean', 'start_idx': 0,..."
2,"Before Persephone was released to Hermes, who ...",When Demeter went to the Underworld to rescue ...,"[a period each year, six or three, Persephone,...","[Underworld, Persephone, Demeter, Hades]","[Persephone, underworld, Hades]",B2,"[{'id': 1, 'text': 'Before', 'start_idx': 0, '...",C1,"[{'id': 1, 'text': 'When', 'start_idx': 0, 'en..."
3,Cogeneration plants are commonly found in dist...,Cogeneration plants are commonly found in dist...,[Cogeneration],[Cogeneration],[Cogeneration],B2,"[{'id': 1, 'text': 'Cogeneration', 'start_idx'...",B2,"[{'id': 1, 'text': 'Cogeneration', 'start_idx'..."
4,"Geneva (,;,;,;;) is the second-most-populous c...",The city's main newspaper is the Tribune de Ge...,"[Switzerland, Romandie, Geneva, second, Zürich...","[the Tribune de Genève, 1 February 1879, James...",[city],B1,"[{'id': 1, 'text': 'Geneva', 'start_idx': 0, '...",C1,"[{'id': 1, 'text': 'The', 'start_idx': 0, 'end..."
...,...,...,...,...,...,...,...,...,...
249161,These structures form at the high-pressure dep...,These structures form at the high-pressure dep...,[Lake Vostok],[Lake Vostok],[Lake Vostok],C2,"[{'id': 1, 'text': 'These', 'start_idx': 0, 'e...",C2,"[{'id': 1, 'text': 'These', 'start_idx': 0, 'e..."
249162,Saint-Maixant is a commune in the Gironde depa...,"Saint-Maixant, Gironde is a commune. It is fou...","[Saint-Maixant, Aquitaine in, Gironde, France]","[Aquitaine in the Gironde, Saint-Maixant, France]","[ne in the Gironde, Saint-Maixant, Aquitaine i...",C2,"[{'id': 1, 'text': 'Saint', 'start_idx': 0, 'e...",C2,"[{'id': 1, 'text': 'Saint', 'start_idx': 0, 'e..."
249163,The Hollywood Walk of Fame is a series of side...,There are many stars on the sidewalk.,"[Vine Street in Hollywood, Hollywood Walk of F...",[],[walk],B2,"[{'id': 1, 'text': 'The', 'start_idx': 0, 'end...",A1,"[{'id': 1, 'text': 'There', 'start_idx': 0, 'e..."
249164,Boulogne-Billancourt is the most populous subu...,"Consequently, Boulogne-Billancourt is one of t...","[Boulogne-Billancourt, Europe, Paris, one]","[Boulogne-Billancourt, Europe]","[Boulogne-Billancourt, Europe]",C1,"[{'id': 1, 'text': 'Boulogne', 'start_idx': 0,...",C2,"[{'id': 1, 'text': 'Consequently', 'start_idx'..."


In [ ]:
def get_word_lists(row,difficulty_level_list_like):
  C2_word_list = []
  # print(row['complex'])
  # print(row['complex_words'])
  if not isinstance(row['complex_words'], float):

    for words in row['complex_words']:
      # print(words['read_level'],words['text'])
      if words['read_level'] in difficulty_level_list_like:
        # print(words['text'])
        if words['text'].lower() not in row['simple'].lower():
          
          if all(words['text'] not in x for x in row['complex_NER']):
            C2_word_list.append(words['text'])

  if len(C2_word_list) >= 1:
    return C2_word_list
  else:
    return np.nan

In [ ]:
C2C1B2_word_lists = []
for index,row in FINAL_df.iterrows():
  # print(index)
  C2C1B2_word_lists.append(get_word_lists(row,['C2','C1','B2']))

In [ ]:
FINAL_df['C2C1B2_in_complex'] = C2C1B2_word_lists

In [ ]:
C2C1B2_df = FINAL_df.drop(columns=['complex_words','simple_words']).dropna()

In [ ]:
C2C1B2_df

,complex,simple,complex_NER,simple_NER,co_occur_NER,complex_read_level,simple_read_level,C2C1B2_in_complex
0,There is manuscript evidence that Austen conti...,There is some proof that Austen continued to w...,"[James Edward Austen, the period 1809, Anna, 1...","[James Edward, Anna Austen, 1814]","[James Edward, Austen, Anna, 1814]",C1,B2,"[manuscript, evidence]"
1,"In a remarkable comparative analysis, Mandaean...",Mandaean scholar Säve-Söderberg showed that Ma...,"[Psalms of Thomas, Säve-Söderberg, Mani's Psal...","[Psalms of Thomas, Säve-Söderberg, Mani's Psal...","[Psalms of Thomas, Säve-Söderberg, Mani's Psal...",B2,B2,"[comparative, analysis]"
2,"Before Persephone was released to Hermes, who ...",When Demeter went to the Underworld to rescue ...,"[a period each year, six or three, Persephone,...","[Underworld, Persephone, Demeter, Hades]","[Persephone, underworld, Hades]",B2,C1,"[released, tricked]"
4,"Geneva (,;,;,;;) is the second-most-populous c...",The city's main newspaper is the Tribune de Ge...,"[Switzerland, Romandie, Geneva, second, Zürich...","[the Tribune de Genève, 1 February 1879, James...",[city],B1,C1,"[populous, populous]"
8,"A scythe (Oxford English Dictionary, Oxford Un...","A scythe (, from Old English siðeOxford Englis...","[Oxford English Dictionary, Oxford University ...","[Oxford English Dictionary, Old English, 1933]","[Oxford English Dictionary, scythe, 1933]",C2,B2,"[agricultural, reaping, crops, reaping, i.e.]"
...,...,...,...,...,...,...,...,...
249157,Barbirolli won a scholarship to study at Trini...,Barbirolli won a scholarship to study at Trini...,"[Sir John Barbirolli Collection of, the Royal ...","[the Royal Academy of Music, Trinity College o...","[the Royal Academy of Music, Trinity College o...",B1,A1,[archived]
249160,"Operationally, the cyclone was first classifie...","On April 21, the subtropical cyclone became a ...","[the National Hurricane Center, Operationally]",[April 21],[],C2,A1,[classified]
249163,The Hollywood Walk of Fame is a series of side...,There are many stars on the sidewalk.,"[Vine Street in Hollywood, Hollywood Walk of F...",[],[walk],B2,A1,"[sidewalks, monument, achievement]"
249164,Boulogne-Billancourt is the most populous subu...,"Consequently, Boulogne-Billancourt is one of t...","[Boulogne-Billancourt, Europe, Paris, one]","[Boulogne-Billancourt, Europe]","[Boulogne-Billancourt, Europe]",C1,C2,[populous]


In [ ]:
with open('/content/drive/MyDrive/muss/qualitative/ABCD_train_C2C1B2_0911', 'wb') as fp:
    pickle.dump(C2C1B2_df, fp)

## validation set

### obtain CERF level using Amplifi engine



In [ ]:
import requests
import json
import ast
from multiprocessing import Pool


In [ ]:
# API
url = #

headers = {
    'Content-Type': 'application/json',
    'x-api-key' : #
}

In [ ]:
# request word level difficulty 
def get_data(text):
    r1 = requests.request("POST", url, headers=headers, data=json.dumps([{'id':'test','text':text}]))
    a = r1.json()
    return a

In [ ]:
with Pool(5) as p:
    result = p.map(get_data, valid_df['complex'])

with open('/content/drive/MyDrive/muss/qualitative/valid_complex_ABCD', 'wb') as fp:
    pickle.dump(result, fp)

In [ ]:
with Pool(5) as p:
    result = p.map(get_data, valid_df['simple'])

with open('/content/drive/MyDrive/muss/qualitative/valid_simple_ABCD', 'wb') as fp:
    pickle.dump(result, fp)

In [ ]:
with open('/content/drive/MyDrive/muss/qualitative/valid_complex_ABCD', 'rb') as f:
    valid_complex_result = pickle.load(f)
with open('/content/drive/MyDrive/muss/qualitative/valid_simple_ABCD', 'rb') as f:
    valid_simple_result = pickle.load(f)

In [ ]:
# combine complex and simple sentences word difficulty level 
from pandas import json_normalize

def process_file(f):
  jsonObj = json.loads(f)
  return json_normalize(jsonObj)

In [ ]:
frames = []

# normalise data if the complex senetnce does not has a corsresponidng result
for i in range(len(valid_complex_result)):

  if 'body' not in valid_complex_result[i].keys():
    print('error',i)
    tmp = process_file(valid_complex_result[i-1]['body'])
    tmp.loc[:] = np.nan
    frames.append(tmp)
  else:
    frames.append(process_file(valid_complex_result[i]['body']))

valid_complex_df = pd.concat(frames)

with open('/content/drive/MyDrive/muss/qualitative/ABCD_valid_complex', 'wb') as fp:
    pickle.dump(valid_complex_df, fp)    

In [ ]:
frames = []

for i in range(len(valid_simple_result)):

  if 'body' not in valid_simple_result[i].keys():
    print('error',i)
    tmp = process_file(valid_simple_result[i-1]['body'])
    tmp.loc[:] = np.nan
    frames.append(tmp)
  else:
    frames.append(process_file(valid_simple_result[i]['body']))

valid_simple_df = pd.concat(frames)

with open('/content/drive/MyDrive/muss/qualitative/ABCD_valid_simple', 'wb') as fp:
    pickle.dump(valid_simple_df, fp)    

In [ ]:
with open('/content/drive/MyDrive/muss/qualitative/ABCD_valid_complex', 'rb') as f:
    valid_complex_df = pickle.load(f)
with open('/content/drive/MyDrive/muss/qualitative/ABCD_valid_simple', 'rb') as f:
    valid_simple_df = pickle.load(f)
    

In [ ]:
new_valid_complex_df = valid_complex_df.drop(columns=['id','read.A1','read.B1','read.B2','read.C1','read.overall']).add_prefix('complex_')
new_valid_simple_df = valid_simple_df.drop(columns=['id','read.A1','read.B1','read.B2','read.C1','read.overall']).add_prefix('simple_')
combined_df = pd.concat([new_valid_complex_df, new_valid_simple_df], axis=1).reset_index().drop(columns='index')
combined_df

,complex_read_level,complex_words,simple_read_level,simple_words
0,C1,"[{'id': 1, 'text': 'Upper', 'start_idx': 0, 'e...",C1,"[{'id': 1, 'text': 'There', 'start_idx': 0, 'e..."
1,A1,"[{'id': 1, 'text': 'It', 'start_idx': 0, 'end_...",B1,"[{'id': 1, 'text': 'It', 'start_idx': 0, 'end_..."
2,A1,"[{'id': 1, 'text': 'Relation', 'start_idx': 0,...",C2,"[{'id': 1, 'text': 'The', 'start_idx': 0, 'end..."
3,C1,"[{'id': 1, 'text': 'A', 'start_idx': 0, 'end_i...",A1,"[{'id': 1, 'text': 'A', 'start_idx': 0, 'end_i..."
4,B1,"[{'id': 1, 'text': 'Of', 'start_idx': 0, 'end_...",A1,"[{'id': 1, 'text': 'The', 'start_idx': 0, 'end..."
...,...,...,...,...
812,A1,"[{'id': 1, 'text': 'Sedimentary', 'start_idx':...",A1,"[{'id': 1, 'text': 'Other', 'start_idx': 0, 'e..."
813,B2,"[{'id': 1, 'text': '""', 'start_idx': 0, 'end_i...",B1,"[{'id': 1, 'text': '""', 'start_idx': 0, 'end_i..."
814,C1,"[{'id': 1, 'text': 'East', 'start_idx': 0, 'en...",C2,"[{'id': 1, 'text': 'The', 'start_idx': 0, 'end..."
815,C2,"[{'id': 1, 'text': 'Certain', 'start_idx': 0, ...",C2,"[{'id': 1, 'text': 'Certain', 'start_idx': 0, ..."


### find C1+C2+B2 words

In [ ]:
sentence_df = valid_df.drop(columns=['level_0','index','complex_NER','simple_NER','co_occur_NER'])
sentence_df

,complex,simple
0,Upper Sorbian is a minority language spoken by...,"There are around 40,000 speakers of Upper Sorb..."
1,It used to be on display as part of the Americ...,It is on permanent display as part of the Amer...
2,Relation to the Nobel Prize The Sveriges Riksb...,The Sveriges Riksbank Prize in Economic Scienc...
3,A shoe is an item of footwear evolved at first...,A shoe is also an item of clothing.
4,Of the eleven tropical cyclones that formed du...,"The season officially started on May 15, 1995 ..."
...,...,...
812,Sedimentary rock is one of the three main rock...,Other types of rock are igneous rock and metam...
813,"""It took more than 60 years after 1859 until t...","""It took more than 60 years after 1859 until t..."
814,"East Timor, also known as Timor-Leste (Tetum: ...",The Democratic Republic of East Timor or Timor...
815,Certain artistic traditions that originated in...,Certain traditions that originated in the Byza...


In [ ]:
FINAL_df = pd.concat([sentence_df, combined_df], axis=1)
FINAL_df = FINAL_df[FINAL_df['complex_words'].notna()][FINAL_df['simple_words'].notna()]
FINAL_df

,complex,simple,complex_read_level,complex_words,simple_read_level,simple_words
0,Upper Sorbian is a minority language spoken by...,"There are around 40,000 speakers of Upper Sorb...",C1,"[{'id': 1, 'text': 'Upper', 'start_idx': 0, 'e...",C1,"[{'id': 1, 'text': 'There', 'start_idx': 0, 'e..."
1,It used to be on display as part of the Americ...,It is on permanent display as part of the Amer...,A1,"[{'id': 1, 'text': 'It', 'start_idx': 0, 'end_...",B1,"[{'id': 1, 'text': 'It', 'start_idx': 0, 'end_..."
2,Relation to the Nobel Prize The Sveriges Riksb...,The Sveriges Riksbank Prize in Economic Scienc...,A1,"[{'id': 1, 'text': 'Relation', 'start_idx': 0,...",C2,"[{'id': 1, 'text': 'The', 'start_idx': 0, 'end..."
3,A shoe is an item of footwear evolved at first...,A shoe is also an item of clothing.,C1,"[{'id': 1, 'text': 'A', 'start_idx': 0, 'end_i...",A1,"[{'id': 1, 'text': 'A', 'start_idx': 0, 'end_i..."
4,Of the eleven tropical cyclones that formed du...,"The season officially started on May 15, 1995 ...",B1,"[{'id': 1, 'text': 'Of', 'start_idx': 0, 'end_...",A1,"[{'id': 1, 'text': 'The', 'start_idx': 0, 'end..."
...,...,...,...,...,...,...
812,Sedimentary rock is one of the three main rock...,Other types of rock are igneous rock and metam...,A1,"[{'id': 1, 'text': 'Sedimentary', 'start_idx':...",A1,"[{'id': 1, 'text': 'Other', 'start_idx': 0, 'e..."
813,"""It took more than 60 years after 1859 until t...","""It took more than 60 years after 1859 until t...",B2,"[{'id': 1, 'text': '""', 'start_idx': 0, 'end_i...",B1,"[{'id': 1, 'text': '""', 'start_idx': 0, 'end_i..."
814,"East Timor, also known as Timor-Leste (Tetum: ...",The Democratic Republic of East Timor or Timor...,C1,"[{'id': 1, 'text': 'East', 'start_idx': 0, 'en...",C2,"[{'id': 1, 'text': 'The', 'start_idx': 0, 'end..."
815,Certain artistic traditions that originated in...,Certain traditions that originated in the Byza...,C2,"[{'id': 1, 'text': 'Certain', 'start_idx': 0, ...",C2,"[{'id': 1, 'text': 'Certain', 'start_idx': 0, ..."


In [ ]:
FINAL_df = pd.concat([valid_df.drop(columns=['level_0','index']), FINAL_df.drop(columns=['complex','simple'])],axis=1)
FINAL_df

,complex,simple,complex_NER,simple_NER,co_occur_NER,complex_read_level,complex_words,simple_read_level,simple_words
0,Upper Sorbian is a minority language spoken by...,"There are around 40,000 speakers of Upper Sorb...","[Sorbs in Germany in the, Upper Sorbian, Upper...",[the European Charter for Regional or Minority...,"[Upper Sorbian, in Germany, inority, Saxony]",C1,"[{'id': 1, 'text': 'Upper', 'start_idx': 0, 'e...",C1,"[{'id': 1, 'text': 'There', 'start_idx': 0, 'e..."
1,It used to be on display as part of the Americ...,It is on permanent display as part of the Amer...,"[Library of Congress in Washington, American T...","[Library of Congress in Washington, American T...","[Library of Congress in Washington, American T...",A1,"[{'id': 1, 'text': 'It', 'start_idx': 0, 'end_...",B1,"[{'id': 1, 'text': 'It', 'start_idx': 0, 'end_..."
2,Relation to the Nobel Prize The Sveriges Riksb...,The Sveriges Riksbank Prize in Economic Scienc...,[Nobel Prize The Sveriges Riksbank Prize in Ec...,[Sveriges Riksbank Prize in Economic Sciences ...,[The Sveriges Riksbank Prize in Economic Scien...,A1,"[{'id': 1, 'text': 'Relation', 'start_idx': 0,...",C2,"[{'id': 1, 'text': 'The', 'start_idx': 0, 'end..."
3,A shoe is an item of footwear evolved at first...,A shoe is also an item of clothing.,[first],[],[],C1,"[{'id': 1, 'text': 'A', 'start_idx': 0, 'end_i...",A1,"[{'id': 1, 'text': 'A', 'start_idx': 0, 'end_i..."
4,Of the eleven tropical cyclones that formed du...,"The season officially started on May 15, 1995 ...","[Hurricane Ismael, at least 116, the season, e...","[the eastern Pacific, November 30, 1995, May 1...","[the season, the e]",B1,"[{'id': 1, 'text': 'Of', 'start_idx': 0, 'end_...",A1,"[{'id': 1, 'text': 'The', 'start_idx': 0, 'end..."
...,...,...,...,...,...,...,...,...,...
812,Sedimentary rock is one of the three main rock...,Other types of rock are igneous rock and metam...,"[Sedimentary, three, one]",[],[],A1,"[{'id': 1, 'text': 'Sedimentary', 'start_idx':...",A1,"[{'id': 1, 'text': 'Other', 'start_idx': 0, 'e..."
813,"""It took more than 60 years after 1859 until t...","""It took more than 60 years after 1859 until t...",[more than 60 years after 1859],[more than 60 years after 1859],[more than 60 years after 1859],B2,"[{'id': 1, 'text': '""', 'start_idx': 0, 'end_i...",B1,"[{'id': 1, 'text': '""', 'start_idx': 0, 'end_i..."
814,"East Timor, also known as Timor-Leste (Tetum: ...",The Democratic Republic of East Timor or Timor...,"[the Democratic Republic of Timor-Leste, South...","[Democratic Republic of East Timor, Southeast ...","[the democratic republic of, Democratic Republ...",C1,"[{'id': 1, 'text': 'East', 'start_idx': 0, 'en...",C2,"[{'id': 1, 'text': 'The', 'start_idx': 0, 'end..."
815,Certain artistic traditions that originated in...,Certain traditions that originated in the Byza...,"[the Byzantine Empire, Eastern Orthodox, the p...","[the Byzantine Empire, Eastern Orthodox, Greec...","[the Byzantine Empire, Eastern Orthodox, Greec...",C2,"[{'id': 1, 'text': 'Certain', 'start_idx': 0, ...",C2,"[{'id': 1, 'text': 'Certain', 'start_idx': 0, ..."


In [ ]:
def get_word_lists(row,difficulty_level_list_like):
  C2_word_list = []
  # print(row['complex'])
  # print(row['complex_words'])
  if not isinstance(row['complex_words'], float):

    for words in row['complex_words']:
      # print(words['read_level'],words['text'])
      if words['read_level'] in difficulty_level_list_like:
        # print(words['text'])
        if words['text'].lower() not in row['simple'].lower():
          
          if all(words['text'] not in x for x in row['complex_NER']):
            C2_word_list.append(words['text'])

  if len(C2_word_list) >= 1:
    return C2_word_list
  else:
    return np.nan

In [ ]:
C2C1B2_word_lists = []
for index,row in FINAL_df.iterrows():
  # print(index)
  C2C1B2_word_lists.append(get_word_lists(row,['C2','C1','B2']))

In [ ]:
len([x for x in C2C1B2_word_lists if str(x) != 'nan'])

418

In [ ]:
FINAL_df['C2C1B2_in_complex'] = C2C1B2_word_lists

In [ ]:
C2C1B2_df = FINAL_df.drop(columns=['complex_words','simple_words']).dropna()

In [ ]:
C2C1B2_df

,complex,simple,complex_NER,simple_NER,co_occur_NER,complex_read_level,simple_read_level,C2C1B2_in_complex
0,Upper Sorbian is a minority language spoken by...,"There are around 40,000 speakers of Upper Sorb...","[Sorbs in Germany in the, Upper Sorbian, Upper...",[the European Charter for Regional or Minority...,"[Upper Sorbian, in Germany, inority, Saxony]",C1,C1,[ica]
3,A shoe is an item of footwear evolved at first...,A shoe is also an item of clothing.,[first],[],[],C1,A1,[evolved]
4,Of the eleven tropical cyclones that formed du...,"The season officially started on May 15, 1995 ...","[Hurricane Ismael, at least 116, the season, e...","[the eastern Pacific, November 30, 1995, May 1...","[the season, the e]",B1,A1,"[tropical, cyclones, affected]"
5,"In 2007/08 it had over 40,000 students studyin...","In 2007-08, it had over 40,000 students studyi...","[the United Kingdom, more than 10,000, over 40...","[more than 10,000, # 637 million, over 40,000,...","[more than 10,000, over 40,000, 2007, 500, 08]",C1,B2,[making]
9,The Summer of Love became a defining moment of...,Thousands of young people from all over the wo...,"[the hippie counterculture movement, Summer of...","[The Summer of Love, San Francisco, Thousands ...","[hippie counterculture, The Summer of Love]",B2,B1,"[defining, into, awareness]"
...,...,...,...,...,...,...,...,...
803,1996 & ndash; Hamas bombmaker Yahya Ayyash is ...,1996 -- Hamas operative Yahya Ayyash is killed...,"[1996 & ndash, Yahya Ayyash, Israeli, Hamas]","[Yahya Ayyash, Israeli, Hamas, 1996]","[Yahya Ayyash, Israeli, Hamas, 1996]",C1,B2,[bombmaker]
804,"She went on tour with the Oyster Band, and the...",She went on tour with the Oyster Band.,"[the following year, the Oyster Band, Rykodisc]",[the Oyster Band],[the Oyster Band],C1,B2,"[limited, promotional]"
805,He would hang out at the Palace Theater in Red...,He helped out at the Palace theater in Red Ban...,"[Palace Theater in Red Bank, the Palace Theater]","[Red Bank, Palace]","[heater in Red Bank, the Palace]",B1,B2,"[hang, chores]"
814,"East Timor, also known as Timor-Leste (Tetum: ...",The Democratic Republic of East Timor or Timor...,"[the Democratic Republic of Timor-Leste, South...","[Democratic Republic of East Timor, Southeast ...","[the democratic republic of, Democratic Republ...",C1,C2,[officially]


In [ ]:
with open('/content/drive/MyDrive/muss/qualitative/ABCD_valid_C2C1B2_0911', 'wb') as fp:
    pickle.dump(C2C1B2_df, fp)

## create dataset

In [ ]:
with open('/content/drive/MyDrive/muss/qualitative/ABCD_train_C2C1B2_0911', 'rb') as f:
    train_C2C1B2_df = pickle.load(f)

with open('/content/drive/MyDrive/muss/qualitative/ABCD_valid_C2C1B2_0911', 'rb') as f:
    valid_C2C1B2_df = pickle.load(f)


In [ ]:
TRAIN_DF = pd.concat([train_df.drop(columns=['level_0','index']),train_C2C1B2_df.drop(columns=['complex','simple','complex_NER','simple_NER','co_occur_NER','complex_read_level','simple_read_level'])], axis=1)
TRAIN_DF

,complex,simple,complex_NER,simple_NER,co_occur_NER,C2C1B2_in_complex
0,There is manuscript evidence that Austen conti...,There is some proof that Austen continued to w...,"[James Edward Austen, the period 1809, Anna, 1...","[James Edward, Anna Austen, 1814]","[James Edward, Austen, Anna, 1814]","[manuscript, evidence]"
1,"In a remarkable comparative analysis, Mandaean...",Mandaean scholar Säve-Söderberg showed that Ma...,"[Psalms of Thomas, Säve-Söderberg, Mani's Psal...","[Psalms of Thomas, Säve-Söderberg, Mani's Psal...","[Psalms of Thomas, Säve-Söderberg, Mani's Psal...","[comparative, analysis]"
2,"Before Persephone was released to Hermes, who ...",When Demeter went to the Underworld to rescue ...,"[a period each year, six or three, Persephone,...","[Underworld, Persephone, Demeter, Hades]","[Persephone, underworld, Hades]","[released, tricked]"
3,Cogeneration plants are commonly found in dist...,Cogeneration plants are commonly found in dist...,[Cogeneration],[Cogeneration],[Cogeneration],NaN
4,"Geneva (,;,;,;;) is the second-most-populous c...",The city's main newspaper is the Tribune de Ge...,"[Switzerland, Romandie, Geneva, second, Zürich...","[the Tribune de Genève, 1 February 1879, James...",[city],"[populous, populous]"
...,...,...,...,...,...,...
249161,These structures form at the high-pressure dep...,These structures form at the high-pressure dep...,[Lake Vostok],[Lake Vostok],[Lake Vostok],NaN
249162,Saint-Maixant is a commune in the Gironde depa...,"Saint-Maixant, Gironde is a commune. It is fou...","[Saint-Maixant, Aquitaine in, Gironde, France]","[Aquitaine in the Gironde, Saint-Maixant, France]","[ne in the Gironde, Saint-Maixant, Aquitaine i...",NaN
249163,The Hollywood Walk of Fame is a series of side...,There are many stars on the sidewalk.,"[Vine Street in Hollywood, Hollywood Walk of F...",[],[walk],"[sidewalks, monument, achievement]"
249164,Boulogne-Billancourt is the most populous subu...,"Consequently, Boulogne-Billancourt is one of t...","[Boulogne-Billancourt, Europe, Paris, one]","[Boulogne-Billancourt, Europe]","[Boulogne-Billancourt, Europe]",[populous]


In [ ]:
VALID_DF = pd.concat([valid_df.drop(columns=['level_0','index']),valid_C2C1B2_df.drop(columns=['complex','simple','complex_NER','simple_NER','co_occur_NER','complex_read_level','simple_read_level'])], axis=1)
VALID_DF

,complex,simple,complex_NER,simple_NER,co_occur_NER,C2C1B2_in_complex
0,Upper Sorbian is a minority language spoken by...,"There are around 40,000 speakers of Upper Sorb...","[Sorbs in Germany in the, Upper Sorbian, Upper...",[the European Charter for Regional or Minority...,"[Upper Sorbian, in Germany, inority, Saxony]",[ica]
1,It used to be on display as part of the Americ...,It is on permanent display as part of the Amer...,"[Library of Congress in Washington, American T...","[Library of Congress in Washington, American T...","[Library of Congress in Washington, American T...",NaN
2,Relation to the Nobel Prize The Sveriges Riksb...,The Sveriges Riksbank Prize in Economic Scienc...,[Nobel Prize The Sveriges Riksbank Prize in Ec...,[Sveriges Riksbank Prize in Economic Sciences ...,[The Sveriges Riksbank Prize in Economic Scien...,NaN
3,A shoe is an item of footwear evolved at first...,A shoe is also an item of clothing.,[first],[],[],[evolved]
4,Of the eleven tropical cyclones that formed du...,"The season officially started on May 15, 1995 ...","[Hurricane Ismael, at least 116, the season, e...","[the eastern Pacific, November 30, 1995, May 1...","[the season, the e]","[tropical, cyclones, affected]"
...,...,...,...,...,...,...
812,Sedimentary rock is one of the three main rock...,Other types of rock are igneous rock and metam...,"[Sedimentary, three, one]",[],[],NaN
813,"""It took more than 60 years after 1859 until t...","""It took more than 60 years after 1859 until t...",[more than 60 years after 1859],[more than 60 years after 1859],[more than 60 years after 1859],NaN
814,"East Timor, also known as Timor-Leste (Tetum: ...",The Democratic Republic of East Timor or Timor...,"[the Democratic Republic of Timor-Leste, South...","[Democratic Republic of East Timor, Southeast ...","[the democratic republic of, Democratic Republ...",[officially]
815,Certain artistic traditions that originated in...,Certain traditions that originated in the Byza...,"[the Byzantine Empire, Eastern Orthodox, the p...","[the Byzantine Empire, Eastern Orthodox, Greec...","[the Byzantine Empire, Eastern Orthodox, Greec...",NaN


In [ ]:
target_dir = '/content/drive/MyDrive/muss/resources/datasets/0911_ABCD_wikilarge_no_inc_na'
!mkdir $target_dir

In [ ]:
data_names = ['train_complex', 'valid_complex']

In [ ]:
df_list = ['train_C2C1B2_df','valid_C2C1B2_df']

In [ ]:
for index,data_name in enumerate(data_names):

  chosen_df = globals()[df_list[index]]
  file_name = data_name.replace('_','.')

  with open(target_dir+'/'+file_name, 'w') as f:
    for index, row in chosen_df.iterrows():

        sentence = row.complex
        NER = row.C2C1B2_in_complex

        if isinstance(NER, float):
          sentence_w_control_token = " <NEXT_DIFFICULT_WORD> "+" <SENT_START> " + sentence
        elif len(NER) == 0:
          sentence_w_control_token = " <NEXT_DIFFICULT_WORD> "+" <SENT_START> " + sentence
        else:
          sentence_w_control_token = " <NEXT_DIFFICULT_WORD> " +" <NEXT_DIFFICULT_WORD> ".join(NER) + " <SENT_START> " + sentence
          

        if index%10000==0:
            print(index)
            print(sentence_w_control_token)

        f.write("{}\n".format(sentence_w_control_token))

        # break

  f.close()


0
 <NEXT_DIFFICULT_WORD> manuscript <NEXT_DIFFICULT_WORD> evidence <SENT_START> There is manuscript evidence that Austen continued to work on these pieces as late as the period 1809 â "11, and that her niece and nephew, Anna and James Edward Austen, made further additions as late as 1814.
10000
 <NEXT_DIFFICULT_WORD> 's <NEXT_DIFFICULT_WORD> watchful <NEXT_DIFFICULT_WORD> issuing <NEXT_DIFFICULT_WORD> plaudits <NEXT_DIFFICULT_WORD> borrow <NEXT_DIFFICULT_WORD> reconstruction <NEXT_DIFFICULT_WORD> capita <SENT_START> In 1992, he became a prime minister, under Syria's watchful and military occupation of Lebanon, he put the country back on the financial map through the issuing of Eurobonds and won plaudits from the World Bank for his plan to borrow reconstruction money as the country's debt grew to become the largest per capita in the world.
40000
 <NEXT_DIFFICULT_WORD> devised <SENT_START> Braille was devised in 1821 by Louis Braille, a blind Frenchman.
50000
 <NEXT_DIFFICULT_WORD> inter

In [ ]:
data_names = ['train_simple', 'valid_simple']

In [ ]:
# train.simple & valid.simple
for index,data_name in enumerate(data_names):

  chosen_df = globals()[df_list[index]]
  file_name = data_name.replace('_','.')
  
  with open(target_dir+'/'+file_name, 'w') as f:

    for index, row in chosen_df.iterrows():

        sentence = row.simple

        if index%10000==0:
            print(index)
            print(sentence)

        f.write("{}\n".format(sentence))

        # break

  f.close()


0
There is some proof that Austen continued to work on these pieces later in life. Her nephew and niece, James Edward and Anna Austen, may have made further additions to her work in around 1814.
10000
He was also the Prime Minister of Lebanon from 1992 to 1998, and he became Prime Minister again in 2000.
40000
To make it possible to read faster, certain transcriptions of Braille use contractions.
50000
The Fairly OddParents became a huge hit, second only in the ratings to SpongeBob SquarePants.
60000
Christians believe that through faith in God they are spiritually made alive with Jesus so that they may lead a new life.
80000
It and its compounds are used mainly in science research though.
110000
Above all, it was the superhighway of transport in ancient times, allowing for trade and cultural exchange between peoples of the region -- Phoenicians, Egyptians, Carthaginians, Greeks, Romans, and the Middle East (Arab\/Persian\/Semitic) cultures.
120000
is about 1 kilometre in diameter, and

# opeartion pererving + lexical simplifition


In [ ]:
train_df = pd.read_pickle("/content/drive/MyDrive/muss/qualitative/train_df_0911")
valid_df = pd.read_pickle("/content/drive/MyDrive/muss/qualitative/valid_df_0911")
test_df = pd.read_pickle("/content/drive/MyDrive/muss/qualitative/test_df_0911")

In [ ]:
with open('/content/drive/MyDrive/muss/qualitative/ABCD_train_C2C1B2_0911', 'rb') as f:
    train_C2C1B2_df = pickle.load(f)

with open('/content/drive/MyDrive/muss/qualitative/ABCD_valid_C2C1B2_0911', 'rb') as f:
    valid_C2C1B2_df = pickle.load(f)


In [ ]:
TRAIN_DF = pd.concat([train_df.drop(columns=['level_0','index']),train_C2C1B2_df.drop(columns=['complex','simple','complex_NER','simple_NER','co_occur_NER','complex_read_level','simple_read_level'])], axis=1)
TRAIN_DF

,complex,simple,complex_NER,simple_NER,co_occur_NER,C2C1B2_in_complex
0,There is manuscript evidence that Austen conti...,There is some proof that Austen continued to w...,"[James Edward Austen, the period 1809, Anna, 1...","[James Edward, Anna Austen, 1814]","[James Edward, Austen, Anna, 1814]","[manuscript, evidence]"
1,"In a remarkable comparative analysis, Mandaean...",Mandaean scholar Säve-Söderberg showed that Ma...,"[Psalms of Thomas, Säve-Söderberg, Mani's Psal...","[Psalms of Thomas, Säve-Söderberg, Mani's Psal...","[Psalms of Thomas, Säve-Söderberg, Mani's Psal...","[comparative, analysis]"
2,"Before Persephone was released to Hermes, who ...",When Demeter went to the Underworld to rescue ...,"[a period each year, six or three, Persephone,...","[Underworld, Persephone, Demeter, Hades]","[Persephone, underworld, Hades]","[released, tricked]"
3,Cogeneration plants are commonly found in dist...,Cogeneration plants are commonly found in dist...,[Cogeneration],[Cogeneration],[Cogeneration],NaN
4,"Geneva (,;,;,;;) is the second-most-populous c...",The city's main newspaper is the Tribune de Ge...,"[Switzerland, Romandie, Geneva, second, Zürich...","[the Tribune de Genève, 1 February 1879, James...",[city],"[populous, populous]"
...,...,...,...,...,...,...
249161,These structures form at the high-pressure dep...,These structures form at the high-pressure dep...,[Lake Vostok],[Lake Vostok],[Lake Vostok],NaN
249162,Saint-Maixant is a commune in the Gironde depa...,"Saint-Maixant, Gironde is a commune. It is fou...","[Saint-Maixant, Aquitaine in, Gironde, France]","[Aquitaine in the Gironde, Saint-Maixant, France]","[ne in the Gironde, Saint-Maixant, Aquitaine i...",NaN
249163,The Hollywood Walk of Fame is a series of side...,There are many stars on the sidewalk.,"[Vine Street in Hollywood, Hollywood Walk of F...",[],[walk],"[sidewalks, monument, achievement]"
249164,Boulogne-Billancourt is the most populous subu...,"Consequently, Boulogne-Billancourt is one of t...","[Boulogne-Billancourt, Europe, Paris, one]","[Boulogne-Billancourt, Europe]","[Boulogne-Billancourt, Europe]",[populous]


In [ ]:
VALID_DF = pd.concat([valid_df.drop(columns=['level_0','index']),valid_C2C1B2_df.drop(columns=['complex','simple','complex_NER','simple_NER','co_occur_NER','complex_read_level','simple_read_level'])], axis=1)
VALID_DF

,complex,simple,complex_NER,simple_NER,co_occur_NER,C2C1B2_in_complex
0,Upper Sorbian is a minority language spoken by...,"There are around 40,000 speakers of Upper Sorb...","[Sorbs in Germany in the, Upper Sorbian, Upper...",[the European Charter for Regional or Minority...,"[Upper Sorbian, in Germany, inority, Saxony]",[ica]
1,It used to be on display as part of the Americ...,It is on permanent display as part of the Amer...,"[Library of Congress in Washington, American T...","[Library of Congress in Washington, American T...","[Library of Congress in Washington, American T...",NaN
2,Relation to the Nobel Prize The Sveriges Riksb...,The Sveriges Riksbank Prize in Economic Scienc...,[Nobel Prize The Sveriges Riksbank Prize in Ec...,[Sveriges Riksbank Prize in Economic Sciences ...,[The Sveriges Riksbank Prize in Economic Scien...,NaN
3,A shoe is an item of footwear evolved at first...,A shoe is also an item of clothing.,[first],[],[],[evolved]
4,Of the eleven tropical cyclones that formed du...,"The season officially started on May 15, 1995 ...","[Hurricane Ismael, at least 116, the season, e...","[the eastern Pacific, November 30, 1995, May 1...","[the season, the e]","[tropical, cyclones, affected]"
...,...,...,...,...,...,...
812,Sedimentary rock is one of the three main rock...,Other types of rock are igneous rock and metam...,"[Sedimentary, three, one]",[],[],NaN
813,"""It took more than 60 years after 1859 until t...","""It took more than 60 years after 1859 until t...",[more than 60 years after 1859],[more than 60 years after 1859],[more than 60 years after 1859],NaN
814,"East Timor, also known as Timor-Leste (Tetum: ...",The Democratic Republic of East Timor or Timor...,"[the Democratic Republic of Timor-Leste, South...","[Democratic Republic of East Timor, Southeast ...","[the democratic republic of, Democratic Republ...",[officially]
815,Certain artistic traditions that originated in...,Certain traditions that originated in the Byza...,"[the Byzantine Empire, Eastern Orthodox, the p...","[the Byzantine Empire, Eastern Orthodox, Greec...","[the Byzantine Empire, Eastern Orthodox, Greec...",NaN


In [ ]:
target_dir = '/content/drive/MyDrive/muss/resources/datasets/0913_ABCD_NER_wikilarge'
!mkdir $target_dir

In [ ]:
data_names = ['train_complex', 'valid_complex']

In [ ]:
df_list = ['TRAIN_DF','VALID_DF']

In [ ]:
for index,data_name in enumerate(data_names):

  chosen_df = globals()[df_list[index]]
  file_name = data_name.replace('_','.')

  
  with open(target_dir+'/'+file_name, 'w') as f:

    for index, row in chosen_df.iterrows():

        sentence = row.complex
        NER = row.simple_NER
        hard_word = row.C2C1B2_in_complex

        if not isinstance(hard_word, float):

          sentence_w_control_token = " <NEXT_NE> "+" <NEXT_NE> ".join(NER)+" <NEXT_DIFFICULT_WORD> "+" <NEXT_DIFFICULT_WORD> ".join(hard_word)+ " <SENT_START> " + sentence


          if index%10000==0:
              print(index)
              print(sentence_w_control_token)

          f.write("{}\n".format(sentence_w_control_token))

        # break

  f.close()


0
 <NEXT_NE> James Edward <NEXT_NE> Anna Austen <NEXT_NE> 1814 <NEXT_DIFFICULT_WORD> manuscript <NEXT_DIFFICULT_WORD> evidence <SENT_START> There is manuscript evidence that Austen continued to work on these pieces as late as the period 1809 â "11, and that her niece and nephew, Anna and James Edward Austen, made further additions as late as 1814.
10000
 <NEXT_NE> Prime Minister of Lebanon <NEXT_NE> 1992 <NEXT_NE> 1998 <NEXT_NE> 2000 <NEXT_DIFFICULT_WORD> 's <NEXT_DIFFICULT_WORD> watchful <NEXT_DIFFICULT_WORD> issuing <NEXT_DIFFICULT_WORD> plaudits <NEXT_DIFFICULT_WORD> borrow <NEXT_DIFFICULT_WORD> reconstruction <NEXT_DIFFICULT_WORD> capita <SENT_START> In 1992, he became a prime minister, under Syria's watchful and military occupation of Lebanon, he put the country back on the financial map through the issuing of Eurobonds and won plaudits from the World Bank for his plan to borrow reconstruction money as the country's debt grew to become the largest per capita in the world.
40000
 <

In [ ]:
data_names = ['train_simple', 'valid_simple']

In [ ]:
# train.simple & valid.simple
for index,data_name in enumerate(data_names):

  chosen_df = globals()[df_list[index]]
  file_name = data_name.replace('_','.')
  
  with open(target_dir+'/'+file_name, 'w') as f:

    for index, row in chosen_df.iterrows():

        sentence = row.simple
        hard_word = row.C2C1B2_in_complex

        if not isinstance(hard_word, float):

          if index%10000==0:
              print(index)
              print(sentence)

          f.write("{}\n".format(sentence))

        # break

  f.close()


0
There is some proof that Austen continued to work on these pieces later in life. Her nephew and niece, James Edward and Anna Austen, may have made further additions to her work in around 1814.
10000
He was also the Prime Minister of Lebanon from 1992 to 1998, and he became Prime Minister again in 2000.
40000
To make it possible to read faster, certain transcriptions of Braille use contractions.
50000
The Fairly OddParents became a huge hit, second only in the ratings to SpongeBob SquarePants.
60000
Christians believe that through faith in God they are spiritually made alive with Jesus so that they may lead a new life.
80000
It and its compounds are used mainly in science research though.
110000
Above all, it was the superhighway of transport in ancient times, allowing for trade and cultural exchange between peoples of the region -- Phoenicians, Egyptians, Carthaginians, Greeks, Romans, and the Middle East (Arab\/Persian\/Semitic) cultures.
120000
is about 1 kilometre in diameter, and